In [1]:
#!pip install transformers datasets torch
import datasets
import transformers
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
import torch
#!pip install transformers[torch]
#!pip install accelerate -U

In [9]:
# Load the dataset
dataset = load_dataset("mehdiiraqui/twitter_disaster")

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize and encode function
def tokenize_and_encode(examples):
    tokenized_inputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    tokenized_inputs['labels'] = examples['target']
    return tokenized_inputs

# Apply the function to the entire dataset
tokenized_datasets = dataset.map(tokenize_and_encode, batched=True)

# Split the dataset (if necessary)
# Example: Create a train and validation split
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['test']  # Or 'validation' if it exists



Map:   0%|          | 0/7613 [00:00<?, ? examples/s]

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [10]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)  # Adjust num_labels as needed


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)


In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],  # or validation set if available
    tokenizer=tokenizer
)


In [13]:
trainer.train()


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [7]:
# Check if labels are present
print(dataset['train'].column_names)


['id', 'keyword', 'location', 'text', 'target']
